## 参数初始化
##### 参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。
##### PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么你需要定义模型中的所有参数，当然这样你可以非常方便地定义每个变量的初始化方式，但是对于复杂的模型，这并不容易，而且我们推崇使用 Sequential 和 Module 来定义模型，所以这个时候我们就需要知道如何来自定义初始化方式

## 1、使用Numpy来初始化
##### 因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import torch
from torch import nn
import numpy as np

In [2]:
# 定义一个Sequential模型
net1 = nn.Sequential(
       nn.Linear(30, 40),
       nn.ReLU(),
       nn.Linear(40, 50),
       nn.ReLU(),
       nn.Linear(50, 10)
       )

In [3]:
net1

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
)

In [4]:
# 查看第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias
# 网络中的参数都是nn.Parameter()，是一种特殊的Variable，即默认是需要求导数的

In [5]:
print(w1)

Parameter containing:
tensor([[ 0.0331, -0.1219,  0.1469,  ..., -0.0536,  0.0059,  0.1165],
        [-0.0684, -0.0087,  0.0364,  ...,  0.1624,  0.0617, -0.0660],
        [-0.1571,  0.0404,  0.1752,  ...,  0.0131, -0.0595,  0.0866],
        ...,
        [-0.1615,  0.1248, -0.1218,  ..., -0.1498, -0.1453, -0.1762],
        [-0.1583,  0.1398, -0.0804,  ..., -0.1393,  0.0470, -0.0477],
        [ 0.0690,  0.1002, -0.1329,  ...,  0.0636,  0.0453,  0.1123]],
       requires_grad=True)


In [6]:
# 由于网络中的参数都是特殊的Variable，那么我们可以通过.data属性得到参数的值，然后用numpy
# 生成随机数据赋值给参数
np_rand_data = np.random.uniform(3., 5., size=(40, 30))  # 均匀分布
net1[0].weight.data = torch.from_numpy(np_rand_data)
print(net1[0].weight)

Parameter containing:
tensor([[4.8943, 4.1553, 3.5397,  ..., 3.6289, 3.4489, 3.0830],
        [3.1137, 4.4197, 4.6257,  ..., 4.3372, 4.2383, 4.2666],
        [3.1731, 4.3207, 4.2652,  ..., 3.3375, 4.6397, 3.5248],
        ...,
        [3.0667, 4.8634, 4.5213,  ..., 4.3973, 3.5438, 4.1092],
        [3.9739, 4.5707, 3.6487,  ..., 3.5165, 4.2935, 3.2860],
        [4.2309, 4.8080, 4.3229,  ..., 4.5099, 3.9480, 4.0206]],
       dtype=torch.float64, requires_grad=True)


In [7]:
# 如果要对相同类型的层初始化成相同的方法，那么可以使用循环去修改，比如：
for layer in net1:
    if isinstance(layer, nn.Linear):  # 判断layer是否是线性层
        param_shape = layer.weight.shape
        np_rand_data = np.random.normal(0., 0.5, size=param_shape)  # 正态分布
        layer.weight.data = torch.from_numpy(np_rand_data)

##### 一种非常流行的初始化方法叫xavier，它使得每一层的输出方差是尽可能相等的，它的公式如下：$$w \in Uniform[-\frac{\sqrt{6}}{\sqrt{n_j+n_{j+1}}},\frac{\sqrt{6}}{\sqrt{n_j+n_{j+1}}}]$$
##### 其中，$n_j$和$n_{j+1}$表示该层的输入输出数目

## 2、torch.nn.init
##### 因为 PyTorch 灵活的特性，我们可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 torch.nn.init，其操作层面仍然在 Tensor 上，下面我们举例说明

In [8]:
from torch.nn import init

In [9]:
print(net1[0].weight)

Parameter containing:
tensor([[ 0.1671, -0.1430, -0.3881,  ..., -0.5399,  1.0349, -0.5522],
        [-0.4452, -0.9012,  0.9934,  ..., -0.1720, -0.0565, -0.8161],
        [-0.4803,  0.5743,  0.2428,  ...,  0.5651, -0.5667, -0.0779],
        ...,
        [ 0.1611,  0.9956,  0.5266,  ...,  0.5243, -0.3774,  0.1172],
        [ 0.2765, -1.2508,  0.4880,  ...,  0.0269, -0.2963, -0.7837],
        [-1.0220,  0.0294, -0.4575,  ..., -0.5793,  0.8551,  0.3066]],
       dtype=torch.float64, requires_grad=True)


In [10]:
# 使用xavier方法进行初始化
init.xavier_uniform_(net1[0].weight.data)
print(net1[0].weight)

Parameter containing:
tensor([[ 0.2020,  0.2322, -0.2746,  ..., -0.1883, -0.2115,  0.1339],
        [-0.2157, -0.0620,  0.0172,  ..., -0.0432,  0.1808,  0.0088],
        [-0.0175, -0.0047, -0.1679,  ...,  0.2047, -0.1618, -0.2337],
        ...,
        [ 0.1507,  0.1593, -0.0531,  ...,  0.0566, -0.0194, -0.0490],
        [ 0.1032,  0.2228,  0.1555,  ...,  0.0892,  0.2559, -0.0635],
        [ 0.1922, -0.2585,  0.2570,  ...,  0.1888, -0.1744, -0.0848]],
       dtype=torch.float64, requires_grad=True)
